Justin Dano <br>
FE550 - Data Visualization Applications<br>
Assignment #3<br>
Due 10/11/2017<br>

#  Cryptocurrency Correlation 

Before building an investment strategy on any type of cryptocurrency, it is important to understand how cryptocurrencies move with other finanial assets. When comparing different finnaical assets it is important that the assets have somewhat similiar economic characteristics. Cryptocurrencies, along with fiat currencies can be classified as 'Store of Value Assets', since their value is derived solely on what humans believe it is worth<sup>1</sup>. In this analysis, I aim to answer the following questions:


<b> 1. What is the correlation between Bitcoin and the main fiat currencies? </b><br>
<b> 2. How has the correlation (or lack thereof) between Bitcoin and other fiat currencies changed over time? </b><br>
<b> 3. What about other assets?  </b>

## 1. Introduction



#### Technology Stack
Python 3.6.1 <br>
Anaconda 3-4.4.0 <br>
Pandas 0.20.3 <br>
Bokeh 0.12.9 <br>
Developed on a Jupyter notebook.

## 2 Get Data

In [19]:
import pandas as pd
import numpy as np
import warnings
import pandas_datareader.data as web
import datetime
from bokeh.plotting import figure
from bokeh.io import output_notebook, gridplot, output_file, show
from bokeh.models import LinearAxis, Range1d, ColumnDataSource, NumeralTickFormatter, DatetimeTickFormatter, Span
from bokeh.layouts import widgetbox, row
from bokeh.models.widgets import TextInput, Button, Paragraph
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application

warnings.filterwarnings('ignore')
output_notebook()

Loading BokehJS ...

## Part 1 - USD/BTC Correlation

In [20]:
# From Bitstamp
bitcoin_data = pd.read_csv('data/BCHARTS-BITSTAMPUSD.csv')
# United States Greenback proxied with UUP ETF
usd_data = pd.read_csv('data/UUP.csv')

bitcoin_data.head()

,Date,Open,High,Low,Close,Volume (BTC),Volume (Currency),Weighted Price
0,2017-10-13,5444.00,5846.43,5380.00,5622.64,27289.973018,1.528179e+08,5599.780647
1,2017-10-12,4822.01,5445.00,4793.66,5445.00,19916.228020,1.025599e+08,5149.565627
2,2017-10-11,4747.90,4869.78,4700.00,4822.01,9866.895622,4.725632e+07,4789.380290
3,2017-10-10,4761.67,4909.97,4700.00,4749.29,11666.024962,5.608879e+07,4807.874965
4,2017-10-09,4597.97,4865.00,4541.00,4764.70,12923.666617,6.047321e+07,4679.260854


In [21]:
usd_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2007-03-01,24.930000,25.000000,24.930000,24.969999,24.598032,4600
1,2007-03-02,25.020000,25.260000,24.950001,24.959999,24.588179,2400
2,2007-03-05,25.110001,25.129999,25.100000,25.120001,24.745798,1100
3,2007-03-06,25.100000,25.100000,25.100000,25.100000,24.726095,400
4,2007-03-07,25.080000,25.080000,24.990000,25.000000,24.627583,2800


In [22]:
def clean_currency_data(curr_df, start_date, bitcoin_flag=False):
    # Set Index as Date
    curr_df = curr_df.set_index('Date')

    # Convert index from string to Date
    curr_df.index = curr_df.index.to_datetime()

    # Select only subset of days based on available history of Bitcoin
    curr_df = curr_df.loc[start_date:]

    # Create new DataFrame, since we are only interested in the Adjusted Close
    temp_df = pd.DataFrame(index=curr_df.index, columns={'price'})

    if bitcoin_flag:
        temp_df['price'] = curr_df['Weighted Price']
    else:
        temp_df['price'] = curr_df['Adj Close']

    return temp_df


def clean_bitcoin_data(btc_df):

    # Reverse dataframe since .csv was originally date descending
    bitcoin_df = btc_df.iloc[::-1]

    # Get First date of available bitcoin data
    first_day = bitcoin_df['Date'].iloc[0]

    # Perform other techniques to clean data, such as setting index
    bitcoin_df = clean_currency_data(bitcoin_df, first_day, True)

    # Fill zero values with preceding price. This keeps the rolling correlation
    # from returning NaN values
    bitcoin_df['price'] = bitcoin_df['price'].replace(to_replace=0, method='ffill')

    return bitcoin_df, first_day

# Clean Bitcoin data and get first trading day
bitcoin_data, first_day = clean_bitcoin_data(bitcoin_data)
usd_data = clean_currency_data(usd_data, first_day)
bitcoin_data.head()

,price
2011-09-13,5.929231
2011-09-14,5.590798
2011-09-15,5.094272
2011-09-16,4.854515
2011-09-17,4.870000


In [23]:
def calculate_log_returns(curr_df):
    curr_df['log_return'] = np.log(curr_df['price']) - np.log(curr_df.price.shift(1))
    return curr_df

bitcoin_df = calculate_log_returns(bitcoin_data)
usd_df = calculate_log_returns(usd_data)
bitcoin_df.head()

,price,log_return
2011-09-13,5.929231,NaN
2011-09-14,5.590798,-0.058773
2011-09-15,5.094272,-0.093005
2011-09-16,4.854515,-0.048208
2011-09-17,4.870000,0.003185


In [24]:
def calculate_rolling_correlation(curr_df, btc_df):
    curr_df['btc_price'] = btc_df['price']
    curr_df['btc_log_return'] = btc_df['log_return']

    curr_df['corr'] = pd.rolling_corr(curr_df['log_return'], curr_df['btc_log_return'], window=254)#, min_periods=1)
    curr_df = curr_df.dropna()
    return curr_df

corr_data = calculate_rolling_correlation(usd_df, bitcoin_df)
corr_data.head()

,price,log_return,btc_price,btc_log_return,corr
2012-09-14,21.639999,-0.005070,11.335450,0.006149,-0.075012
2012-09-17,21.670000,0.001385,11.736280,0.006019,-0.077323
2012-09-18,21.740000,0.003225,11.872507,0.011540,-0.084845
2012-09-19,21.670000,-0.003225,12.309387,0.036137,-0.083640
2012-09-20,21.770000,0.004604,12.429165,0.009684,-0.083907


In [25]:
p_height = 400
p_width = 800

# Horizontal line
hline = Span(location=0, dimension='width', line_color='black', line_width=1)

p = figure(plot_width=p_width, plot_height=p_height, title='United States and Bitcoin Correlation',
           x_axis_label='Time', x_axis_type='datetime',
           y_axis_label='Correlation', y_range=(-0.5, 0.5))
p.line(corr_data.index, corr_data['corr'], line_width=2)
p.renderers.extend([hline])
show(p)

## Part 2 - Leading Currencies Correlation

In [26]:
def clean_and_calculate_corr(curr_df, btc_df, start_date):
    # Clean the data
    curr_df = clean_currency_data(curr_df, start_date)

    # Calculate log returns for the ETF
    curr_df = calculate_log_returns(curr_df)

    # Calculate rolling correlation for
    curr_df = calculate_rolling_correlation(curr_df, btc_df)

    return curr_df

In [27]:
bitcoin_df = pd.read_csv('data/BCHARTS-BITSTAMPUSD.csv')
# United States Greenback proxied with UUP ETF
usd_df = pd.read_csv('data/UUP.csv')
# China's Yuan Renmibni proxied with CYB ETF
cny_df = pd.read_csv('data/CYB.csv')
# Japanese Yen proxied with EWJ ETF
jpy_df = pd.read_csv('data/EWJ.csv')
# Australian Dollar proxied with EWA ETF
aud_df = pd.read_csv('data/EWA.csv')
# Euro proxied with FXE ETF
eur_df = pd.read_csv('data/FXE.csv')
# British Pound proxied with FXB ETF
gbp_df = pd.read_csv('data/FXB.csv')
# Swiss Franc proxied with FXF ETF
chf_df = pd.read_csv('data/FXB.csv')

# Clean Bitcoin data and get first trading day
bitcoin_data, first_day = clean_bitcoin_data(bitcoin_df)
bitcoin_df = calculate_log_returns(bitcoin_data)

# Clean and calculate correlations for other currency ETFs
usd_data = clean_and_calculate_corr(usd_df, bitcoin_df, first_day)
cny_data = clean_and_calculate_corr(cny_df, bitcoin_df, first_day)
jpy_data = clean_and_calculate_corr(jpy_df, bitcoin_df, first_day)
aud_data = clean_and_calculate_corr(aud_df, bitcoin_df, first_day)
eur_data = clean_and_calculate_corr(eur_df, bitcoin_df, first_day)
gbp_data = clean_and_calculate_corr(gbp_df, bitcoin_df, first_day)

eur_data.head()

,price,log_return,btc_price,btc_log_return,corr
2012-09-14,130.399994,0.009942,11.335450,0.006149,0.065972
2012-09-17,130.270004,-0.000997,11.736280,0.006019,0.068970
2012-09-18,129.559998,-0.005465,11.872507,0.011540,0.079339
2012-09-19,129.720001,0.001234,12.309387,0.036137,0.076575
2012-09-20,128.850006,-0.006729,12.429165,0.009684,0.076877


In [28]:
def plot_grid(data_arr, title_arr):
    grid_arr = []
    p_height = 200
    p_width = 400

    # Horizontal line
    hline = Span(location=0, dimension='width', line_color='black', line_width=1)

    for i, curr in enumerate(data_arr):

        p = figure(plot_width=p_width, plot_height=p_height, title=title_arr[i],
                   x_axis_label='Time', x_axis_type='datetime',
                   y_axis_label='Correlation', y_range=(-0.5, 0.5))
        p.line(curr.index, curr['corr'], line_width=2)
        p.min_border_right = 40
        p.renderers.extend([hline])
        grid_arr.append(p)

    gp = gridplot([[grid_arr[0], grid_arr[1]], [grid_arr[2], grid_arr[3]], [grid_arr[4], grid_arr[5]]])

    show(gp)

currencies = [usd_data, cny_data, jpy_data, aud_data, eur_data, gbp_data]
titles = ['United States', 'China', 'Japan', 'Australia', 'Europe', 'Great Britain']
plot_grid(currencies, titles)    

## Part 3 -Interactive Currency Correlations

In [36]:
def pull_web_data(ticker):
    start = datetime.datetime(2011, 9, 13)
    end = datetime.datetime(2017, 10, 14)
   
    try:        
        
        data = web.DataReader(ticker, 'yahoo', start, end)
        
        # Create new DataFrame, since we are only interested in the Adjusted Close
        temp_df = pd.DataFrame(index=data.index, columns={'price'})
        temp_df['price'] = data['Adj Close']
        
        # Calculate log returns for the ETF
        temp_df = calculate_log_returns(temp_df)

        # Calculate rolling correlation for
        temp_df = calculate_rolling_correlation(temp_df, bitcoin_df)        
        
    except:
        error_msg = 'Error: Ticker %s does not exist in Yahoo!', ticker
        return error_msg
    
    return temp_df


# Create the Document Application
def modify_doc(doc):
    
    # Creates the correlation plot
    def create_figure(user_input_ticker):
        
        user_data = pull_web_data(user_input_ticker)
        if isinstance(user_data, pd.DataFrame):
            p_height = 400
            p_width = 800

            # Horizontal line
            hline = Span(location=0, dimension='width', line_color='black', line_width=1)

            p = figure(plot_width=p_width, plot_height=p_height, title= user_input_ticker + ' and Bitcoins Correlation', 
                       x_axis_label='Time', x_axis_type='datetime', 
                       y_axis_label='Correlation', y_range=(-0.5, 0.5))

            p.line(user_data.index, user_data['corr'], line_width=2)
            p.renderers.extend([hline])           
        else:
            p = Paragraph(text="""Ticker was not found in Yahoo!""", width=200, height=100)

        return p
        
    
    # Update the plot
    def update(attr, old, new):
        print('attr is: ' + str(attr))
        print('old is: ' + str(old))
        print('new is: ' + str(new))
        layout.children[1] = create_figure(new)
    
    # Controls based on UI
    text_input = TextInput(value="FXE", title="Choose Ticker:")

    text_input.on_change('value', update)
    controls = widgetbox([text_input], width=200)
    p = create_figure('FXE')
    layout = row(controls, p)
    doc.add_root(layout)

# Set up the Application 
handler = FunctionHandler(modify_doc)
app = Application(handler)

In [37]:
# Create the Document
# Not strictly necessary, but helps w/ debugging
doc = app.create_document()

In [38]:
# Show the application
# Make sure the URL matches your Jupyter instance
show(app, notebook_url="localhost:8888")

References: <br>
    (1) https://seekingalpha.com/article/4047264-bitcoins-correlation-financial-assets<br>
    (2) https://github.com/ecerami/pydata-essentials/blob/master/bokeh/bokeh_notebook_embed.ipynb
